In [1]:
# Load required packages
library(dplyr,warn.conflicts=F)
library(stringr,warn.conflicts=F)
library(psych,warn.conflicts=F)
library(lplyr,warn.conflicts=F)

In [2]:
# load the stats
data.tourney <- readRDS('datafiles_rds/NCAATourneyDetailedResults.rds')
# keep 2014+
data.tourney %>% filter(Season>2013) -> data.tourney

In [3]:
# store 68 tourney teams 
data.tourney %>% 
    select(Season,TeamID=WTeamID) %>% 
    bind_rows(data.tourney %>% 
              select(Season,TeamID=LTeamID)) %>% 
    distinct(Season,TeamID) -> df.tourneyteams

In [56]:
# DunkleIndex Rating 2014+
# http://dunkelindex.com/rankings/basketball/ncaa/
data.dunkle <- read.csv('datafiles_csv/DunkelIndex.csv',stringsAsFactors=F)
data.dunkle %>% head

SEASON,RANK,TEAMS,RATING,RECORD,CONFERENCE
2018,1,Villanova,91.112,36-4,Big East
2018,2,Michigan,87.017,33-8,Big Ten
2018,3,Duke,83.639,29-8,ACC
2018,4,Kansas,82.436,31-8,Big 12
2018,5,Michigan State,80.526,30-5,Big Ten
2018,6,Cincinnati,80.504,31-5,American


In [57]:
data.dunkle %>% rename(Season=SEASON,
                       Rank=RANK,
                       Name=TEAMS,
                       Rating=RATING,
                       Record=RECORD,
                       Conference=CONFERENCE) -> data.dunkle

In [58]:
# load team spellings variations
data.teamspellings <- readRDS('datafiles_rds/TeamSpellings.rds')
data.teams <- readRDS('datafiles_rds/Teams.rds')

In [75]:
# add TeamID based on team name
data.dunkle %>% 
    mutate(LowName=str_to_lower(Name)) %>% 
    left_join(data.teamspellings,by=c('LowName'='TeamNameSpelling')) -> df.dunkle

In [77]:
data.dunkle %>% filter(str_detect(Name,'East\\.{0,1} Washington'))

Season,Rank,Name,Rating,Record,Conference
2018,150,East. Washington,56.265,20-15,Big Sky
2017,174,East Washington,53.600,22-12,Big Sky
2016,199,East. Washington,50.250,18-16,Big Sky
2015,123,East. Washington,58.154,26-9,Big Sky
2014,209,East. Washington,50.047,15-16,Big Sky


In [78]:
# missing tourney team in Dunkle stats (because name misspelling)
df.dunkle %>% 
    right_join(df.tourneyteams,by='TeamID') %>%
    filter(is.na(Rank)) %>%
    select(TeamID) %>%
    inner_join(data.teams,by='TeamID') %>%
    distinct() %>%
    arrange(TeamID)

TeamID,TeamName,FirstD1Season,LastD1Season
1114,Ark Little Rock,1985,2019
1142,Cal Poly SLO,1995,2019
1167,CS Bakersfield,2008,2019
1168,CS Fullerton,1985,2019
1186,E Washington,1985,2019
1192,F Dickinson,1985,2019
1291,Mt St Mary's,1989,2019
1385,St John's,1985,2019
1386,St Joseph's PA,1985,2019
1387,St Louis,1985,2019


In [79]:
# add missing TeamID from Sagarin stats
df.dunkle %>%
    mutate_which(str_detect(Name,'AR-Little Rock'),TeamID=1114) %>%
    mutate_which(str_detect(Name,'CalPoly'),TeamID=1142) %>%
    mutate_which(str_detect(Name,'CS-Bakersfield'),TeamID=1167) %>%
    mutate_which(str_detect(Name,'CS-Fullerton'),TeamID=1168) %>%
    mutate_which(str_detect(Name,'East\\.{0,1} Washington'),TeamID=1186) %>%
    mutate_which(str_detect(Name,'Fair Dickinson'),TeamID=1192) %>%
    mutate_which(str_detect(Name,'Mt. St. Mary'),TeamID=1291) %>%
    mutate_which(str_detect(Name,'St. John'),TeamID=1385) %>%
    mutate_which(str_detect(Name,'St. Louis'),TeamID=1387) %>%
    mutate_which(str_detect(Name,'St. Joseph'),TeamID=1386) %>%
    mutate_which(str_detect(Name,'^St. Mary'),TeamID=1388) %>%
    mutate_which(str_detect(Name,'LA-Lafayette'),TeamID=1418) %>%               # ULL University of Louisiana at Lafayette 
    mutate_which(str_detect(Name,'MD-Baltimore Co.'),TeamID=1420) %>%  # UMBC University Maryland Baltimore County
    mutate_which(str_detect(Name,'UNC-Greensboro'),TeamID=1422) %>%
    mutate_which(str_detect(Name,'UNC-Wilmington'),TeamID=1423) %>%
    mutate_which(str_detect(Name,'WI-Green Bay'),TeamID=1453) %>%
    mutate_which(str_detect(Name,'WI-Milwaukee'),TeamID=1454) -> df.dunkle

In [80]:
# check if ssomething is still wrong
df.dunkle %>% 
    right_join(df.tourneyteams,by='TeamID') %>%
    filter(is.na(Rank)) %>%
    select(TeamID) %>%
    inner_join(data.teams,by='TeamID') %>%
    distinct() %>%
    arrange(TeamID)

TeamID,TeamName,FirstD1Season,LastD1Season


In [81]:
# restrict Dunkle stats to tourney teams
df.dunkle %>% 
    inner_join(df.tourneyteams,by=c('TeamID','Season')) %>%
    select(Season,TeamID,Rating) -> df.dunkle

In [82]:
# Check if we have 68 teams per Season
df.dunkle %>% 
    group_by(Season) %>% 
    count

Season,n
2014,68
2015,68
2016,68
2017,68
2018,68


In [83]:
# check missing TeamID per Season 
df.tourneyteams %>% 
    anti_join(df.dunkle,by=c('Season','TeamID'))

Season,TeamID


In [84]:
# compute the Truth 
data.tourney %>% 
    select(Season,WTeamID,LTeamID) %>% 
    mutate(TeamID.1=ifelse(WTeamID<LTeamID,WTeamID,LTeamID),
           TeamID.2=ifelse(WTeamID>LTeamID,WTeamID,LTeamID)) %>%
    mutate(ID=str_c(Season,'_',TeamID.1,'_',TeamID.2)) %>% 
    mutate(Target=ifelse(WTeamID==TeamID.1,1,0)) %>% 
    select(ID,Target) -> df.truth
df.truth %>% sample_n(5)

ID,Target
2016_1314_1323,1
2016_1173_1393,0
2014_1246_1458,1
2016_1122_1242,0
2017_1242_1413,1


In [86]:
# slope hyper parameter grid : we will compute 51 models and chose the best one (min logloss)
df.dunkle %>% 
    mutate(dummy=1) %>% 
    inner_join(data.frame(qtl=seq(0.25,0.75,0.01),dummy=1),by='dummy') %>% 
    select(-dummy) %>% 
    group_by(qtl,Season) %>% 
    mutate(thres=max(Rating)-quantile(Rating,max(qtl)),Slope=3/thres) %>%
    ungroup %>% 
    select(Season,qtl,Slope) %>% 
    distinct() -> df.dunkle.slopes
df.dunkle.slopes %>% sample_n(5)

Season,qtl,Slope
2016,0.43,0.1560084
2015,0.61,0.1390347
2018,0.33,0.1309766
2016,0.63,0.1905047
2017,0.27,0.1369169


In [87]:
# matchups dataframe : all the possible matches between the 68 teams per Season 
df.dunkle %>% 
    inner_join(df.dunkle,by='Season') %>% 
    filter(TeamID.x < TeamID.y) -> df.matchup
df.matchup %>% sample_n(5)

Season,TeamID.x,Rating.x,TeamID.y,Rating.y
2016,1221,48.732,1277,83.002
2015,1318,57.917,1438,81.546
2017,1321,70.233,1452,79.700
2016,1276,68.325,1433,70.027
2016,1320,68.468,1386,72.148


In [88]:
# all matchs with all 51 hyper parameter setting (qtl=[0.25,0.75]) 
df.matchup %>% inner_join(df.dunkle.slopes,by=c('Season')) -> df.matchup
df.matchup %>% sample_n(5)

Season,TeamID.x,Rating.x,TeamID.y,Rating.y,qtl,Slope
2015,1181,88.472,1361,70.945,0.36,0.1207901
2015,1186,58.154,1414,60.522,0.42,0.1237583
2015,1214,49.280,1329,66.111,0.44,0.1265759
2017,1308,64.644,1411,50.933,0.43,0.1589088
2017,1425,71.316,1462,72.783,0.67,0.2189660


In [89]:
# machups predictions : compute the winning probability of TeamID.x and cap it into [0.05,0.95] to avoid too much logloss is case of FP or FN 
df.matchup %>% 
    mutate(ID=str_c(Season,'_',TeamID.x,'_',TeamID.y)) %>% 
    mutate(Prob=logistic(Rating.x-Rating.y,a=Slope)) %>% 
    mutate(Pred=ifelse(Prob>0.5,1,0)) %>% 
    mutate(Prob=round(Prob,3)) %>%
    mutate_which(Prob>0.95,Prob=0.95) %>%
    mutate_which(Prob<0.05,Prob=0.05) %>%
    select(qtl,ID,Pred,Prob) -> df.submit
df.submit %>% sample_n(5)

qtl,ID,Pred,Prob
0.26,2017_1211_1448,1,0.944
0.59,2016_1393_1437,0,0.056
0.47,2017_1195_1305,0,0.180
0.35,2015_1140_1242,0,0.385
0.44,2017_1116_1314,0,0.134


In [90]:
# merge prediction and truth
df.truth %>% inner_join(df.submit,by='ID') -> df.results

In [91]:
# compute accuracy & Logloss per qtl to find the best one
df.results %>% 
    mutate(OK=ifelse(Pred==Target,1,0)) %>%
    mutate(LogLoss=Target*log(Prob) + (1-Target)*log(1-Prob)) %>%
    group_by(qtl) %>%
    summarise(TC=sum(OK),N=n(),FC=N-TC,Acc=TC/N,LogLoss=-mean(LogLoss)) %>%
    select(qtl,TC,FC,N,Acc,LogLoss) -> df.results.perf
df.results.perf %>% arrange(LogLoss) %>% head(5)

qtl,TC,FC,N,Acc,LogLoss
0.67,269,66,335,0.8029851,0.4503006
0.66,269,66,335,0.8029851,0.4503205
0.68,269,66,335,0.8029851,0.4503670
0.65,269,66,335,0.8029851,0.4503811
0.63,269,66,335,0.8029851,0.4504738


In [92]:
# store the best qtl
df.results.perf %>% 
    arrange(LogLoss) %>% 
    head(1) %>% pull(qtl) -> bestqtl

In [93]:
# save the best results for kaggle submission
df.submit %>% 
    filter(qtl==bestqtl) %>% 
    select(ID,Pred=Prob) %>% 
    write.csv('predictions/Pred_Dunkle_bestqtl.csv',quote=F,row.names=F)

In [97]:
# save best model for mix
df.submit %>%
    filter(qtl==bestqtl) %>% 
    saveRDS('predictions/Dunkle_bestqtl.rds')    